# Mały projekt 1: wizualizacja poziomu zanieczyszczeń (PM2.5)

Główny Inspektorat Ochrony Środowiska (GIOS) udostępnia dane o jakości powietrza w Polsce na stronie [https://powietrze.gios.gov.pl](https://powietrze.gios.gov.pl), tj. poziom stężenia pyłów PM2.5, PM10, SO2 i innych zanieczyszczeń. Dane te są szczególnie przydatne w analizach środowiskowych i zdrowotnych. W tym zadaniu interesują nas godzinne pomiary stężeń drobnego pyłu **PM2.5** (pyłu o średnicy poniżej 2.5 µm) w latach **2014, 2019 i 2024**. Pyły PM2.5 są one bardzo szkodliwe dla zdrowia, gdyż mogąc przenikać głęboko do układu oddechowego i krwiobiegu. Zadanie polega na przeprowadzeniu analizy danych wraz z opisami czynności oraz wykresami.

## Wymagania ogólne

- Rozwiązanie należy przygotować w formie **Jupyter Notebooka (`.ipynb`)**.  Rozwiązanie proszę przesłać przez Moodle’a.
- Wszystkie operacje czyszczenia i łączenia danych wykonaj **programistycznie**, bez ręcznej edycji plików.
- Do każdego punktu należy obowiązkowo dodać opis z analizą otrzymanych wyników.  
- Zadanie należy wykonać w wylosowanych parach. Obie osoby z pary przesyłają **identyczne pliki** i dodają na początku notebooka opis swojego wkładu.
- Ocena jest wspólna dla pary, a nie indywidualna.

## Dane wejściowe i metadane

- Każdy rok to archiwum ZIP dostępne pod adresem [https://powietrze.gios.gov.pl/pjp/archives/](https://powietrze.gios.gov.pl/pjp/archives/).
- Wewnątrz znajdują się pliki Excel, np. `2024_PM25_1g.xlsx`.
- Poniższa funkcja `download_gios_archive` pozwala odczytać odpowiedni zbior danych; resztę danych pomijamy.
- Użyj pliku metadanych (dostępnego na tej samej stronie), aby zaktualizować **kody stacji** (część stacji mogła w międzyczasie zmienić nazwę). W metadanych znajdziesz odpowiednie kolumny.

## Zadania

### 1. Wczytanie i czyszczenie danych

Wczytaj dane dla lat **2014, 2019 i 2024**, oczyścić je z niepotrzebnych wierszy oraz ujednolić ich format. Zaktualizuj stare kody stacji zgodnie z metadanymi. Pozostaw tylko stacje występujące we wszystkich trzech latach. Informacje o stacjach pomiarowych warto uzupełnić o miejscowości dostępne w metadanych, np. za pomocą MultiIndex: (miejscowość, kod stacji). Pomiary dokonane o północy (00:00:00) powinny być potraktowane jako dotyczące poprzedniego dnia. Połącz dane z trzech lat w jeden `DataFrame` i zapisz do pliku.

### 2. Średnie miesięczne + trend dla miast

Oblicz średnie miesięczne stężenie PM2.5 dla każdej stacji i roku. Dla **Warszawy** i **Katowic**, po uśrednieniu po wszystkich stacjach z tych miast, narysuj wykres liniowy pokazujący trend średnich miesięcznych wartości PM2.5 w 2014 i 2024 roku. Oś X - miesiące (1-12); oś Y - średnia wartość PM2.5; 4 linie trendu. Dołącz opis i interpretację obserwowanych różnic.

### 3. Heatmapa miesięcznych średnich

Dla każdej miejscowości przedstaw heatmapę średnich miesięcznych stężeń PM2.5 w latach lat 2014, 2019 i 2024 (oś X – miesiąc, oś Y – rok). Uśrednij wartości po wszystkich stacjach w danej miejscowości. Każdy panel (facet) ma odpowiadać jednej miejscowości. Dołącz interpretację obserwowanych wyników.

### 4. Dni z przekroczeniem normy (WHO)

Dla każdej stacji i roku policz liczbę dni, w których wystąpiło przekroczenie dobowej normy stężenia PM2.5, czyli 15 µg/m³ (źródło: [https://airscan.org/new-who-air-quality-guidelines-2021/](https://airscan.org/new-who-air-quality-guidelines-2021/)). Znajdź 3 stacje z najmniejszą i 3 stacje z największą liczbą dni z przekroczeniem normy dobowej w 2024 roku. Dla tych 6 stacji narysuj *grouped barplot*, gdzie oś X – stacje, oś Y – liczba dni z przekroczeniem, kolor – rok (2014, 2019, 2024). Dołącz opis i interpretację obserwowanych różnic.

## Dodatkowe wymagania i sugestie

- Notebook powinien zawierać *sanity checks*, np.:
  - liczba stacji w każdym pliku,
  - liczba dni w każdym roku,  
  - kilka przykładowych mapowań kodów stacji,  
- Wszystkie wykresy powinny mieć tytuły, legendy i krótki opis interpretacji.
- Zachęcamy do weryfikacji kodu napisanego przez drugą osobę, gdyż ocena jest wspólna.
- Można wykorzystać dowolne poznane biblioteki do analizy i wizualizacji danych w Pythonie.

## Kryteria oceny

- Zadanie 1: 3 pkt
- Zadanie 2: 2 pkt
- Zadanie 3: 1.5 pkt
- Zadanie 4: 2 pkt
- Jakość wyjaśnień, interpretacje, opis wkładu: 1.5 pkt



### Podział pracy:

**Dominika**: Zadania 1 i 3 <br><br>
**Aleksander**: Zadania 2 i 4

In [1]:
import pandas as pd
# import requests
# import zipfile
# import io
# from datetime import datetime, timedelta
# import seaborn as sns
# import matplotlib.pyplot as plt
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# import numpy as np

# from load_data import *
from compute_averages import *
from visualizations import *

# FIXME: usunąć zbędne importy (a niektóre będą zbędne skoro robimy wszystko importowanymi funkcjami)

### Zadanie 1

In [ ]:
# id archiwum dla poszczególnych lat
gios_archive_url = "https://powietrze.gios.gov.pl/pjp/archives/downloadFile/"
gios_url_ids = {2014: '302', 2019: '322', 2024: '582'}
gios_pm25_file = {2014: '2014_PM2.5_1g.xlsx', 2019: '2019_PM25_1g.xlsx', 2024: '2024_PM25_1g.xlsx'}

# funkcja do ściągania podanego archiwum
def download_gios_archive(year, gios_id, filename):
    # Pobranie archiwum ZIP do pamięci
    url = f"{gios_archive_url}{gios_id}"
    response = requests.get(url)
    response.raise_for_status()  # jeśli błąd HTTP, zatrzymaj
    
    # Otwórz zip w pamięci
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        # znajdź właściwy plik z PM2.5
        if not filename:
            print(f"Błąd: nie znaleziono {filename}.")
        else:
            # wczytaj plik do pandas
            with z.open(filename) as f:
                try:
                    df = pd.read_excel(f, header=0)
                except Exception as e:
                    print(f"Błąd przy wczytywaniu {year}: {e}")
    return df

# Przykladowe użycie
df2014 = download_gios_archive(2014, gios_url_ids[2014], gios_pm25_file[2014])
df2019 = download_gios_archive(2019, gios_url_ids[2019], gios_pm25_file[2019])
df2024 = download_gios_archive(2024, gios_url_ids[2024], gios_pm25_file[2024])

In [ ]:
#usuwanie pierwszego wiersza aby ujednolicić format
df2019.columns = df2019.iloc[0].astype(str).str.strip()
df2019 = df2019[1:]

df2024.columns = df2024.iloc[0].astype(str).str.strip()
df2024 = df2024[1:]

In [ ]:
#pobranie metadanych

# AJ 03.01.26: strona nie daje poprawnej odpowiedzi i dlatego pd.read_excel() wyrzuca ValueError
metadata_url = 'https://powietrze.gios.gov.pl/pjp/archives/downloadFile/584'
metadata_response = requests.get(metadata_url)
metadata_response.raise_for_status()

dfmetadata = pd.read_excel(io.BytesIO(metadata_response.content), header=None)

#aktualizacja kodów stacji
new_codes = dict(zip(dfmetadata[4], dfmetadata[1]))

for df in ['df2014', 'df2019', 'df2024']:
    exec(f"{df} = {df}.rename(columns=new_codes)") 

In [ ]:
#pozostawienie w danych tylko tych stacji, które wystepują w we wszystkich latach, czyli 2014, 2019 i 2024
common_stations = (df2014.columns.intersection(df2019.columns).intersection(df2024.columns))

def only_common_stations(df, common):
    station_cols = [c for c in df.columns if c in common]
    return df[station_cols]

df2014 = only_common_stations(df2014, common_stations)
df2019 = only_common_stations(df2019, common_stations)
df2024 = only_common_stations(df2024, common_stations)

In [ ]:
print("Liczba stacji w podanych latach:")
print(f"2014: {len(df2014.columns) - 1}")
print(f"2019: {len(df2019.columns) - 1}")
print(f"2024: {len(df2024.columns) - 1}")

In [ ]:
#dodanie miejscowości do kodów stacji za pomocą MultiIndex
code_city = dict(zip(dfmetadata[1], dfmetadata[11]))

multi_index = pd.MultiIndex.from_tuples([(code_city.get(code, None), code) for code in common_stations], names=["Miejscowość", "Kod stacji"])

df2014.columns = multi_index
df2019.columns = multi_index
df2024.columns = multi_index

In [ ]:
#zmiana dnia pomiaru o północy na poprzedni
def change_midnight(df, start_row):

    df = df.copy()

    for i in range(start_row, len(df)):
        date = df.iloc[i, 0]
        day_hour = datetime.strptime(str(date), "%Y-%m-%d %H:%M:%S")
        if day_hour.hour == 0:
            day_hour -= timedelta(days=1)
        df.iloc[i, 0] = day_hour

    return df

df2014 = change_midnight(df2014, 5)
df2019 = change_midnight(df2019, 5)
df2024 = change_midnight(df2024, 5)

In [ ]:
#dodanie kolumny year aby następnie połączyć wszystkie dane w jeden plik .csv
df2014.loc[:, 'year'] = 2014
df2019.loc[:, 'year'] = 2019
df2024.loc[:, 'year'] = 2024

all_data = pd.concat([df2014, df2019, df2024], ignore_index=True)
all_data.to_csv("all_data.csv", index=False)

### Brudnopis

In [46]:
all_data = pd.read_csv("all_data.csv")
data = all_data.copy()

/tmp/ipykernel_39201/1455597504.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv("all_data.csv")


In [ ]:
data = all_data.copy()

dt = pd.to_datetime(
    data["Kod stacji"],
    format="%Y-%m-%d %H:%M:%S",
    errors="coerce"
)

# unify datetime format (remove miliseconds)

data["year"] = dt.dt.year
data["month"] = dt.dt.month

,Kod stacji,"('Jelenia Góra', 'DsJelGorOgin')","('Wrocław', 'DsWrocAlWisn')","('Wrocław', 'DsWrocWybCon')","('Bydgoszcz', 'KpBydPlPozna')","('Bydgoszcz', 'KpBydWarszaw')","('Lublin', 'LbLubObywate')","('Łódź', 'LdLodzCzerni')","('Zgierz', 'LdZgieMielcz')","('Zielona Góra', 'LuZielKrotka')",...,"('Kościerzyna', 'PmKosTargowa')","('Katowice', 'SlKatoKossut')","('Złoty Potok', 'SlZlotPotLes')","('Olsztyn', 'WmOlsPuszkin')","('Kalisz', 'WpKaliSawick')","('Szczecin', 'ZpSzczAndrze')","('Szczecin', 'ZpSzczPilsud')",Rok,year,month
0,2015-01-01 01:00:00.000,151.112,78,50,29.2,95.5,71.1012,59.73,58.690689,NaN,...,NaN,51.389,49.9464,44.060833,25.5,NaN,NaN,2015,NaN,NaN
1,2015-01-01 02:00:00.000,262.566,42,33.8244,27.1,86.1,95.2582,185.92,44.501255,51.8628,...,NaN,37.709,45.6291,49.943329,32.5,81.8501,215.379,2015,NaN,NaN
2,2015-01-01 03:00:00.005,222.83,27,28.7215,7.9,23.8,49.5078,54.25,24.163937,15.866,...,NaN,41.984,49.2093,20.688448,24.3,25.7333,27.7458,2015,NaN,NaN
3,2015-01-01 04:00:00.010,210.767,24,20.6891,7.1,15.9,60.8468,50.15,19.516039,20.1187,...,22.7895,45.404,53.316,17.437717,24.4,18.624,26.5201,2015,NaN,NaN
4,2015-01-01 05:00:00.015,191.211,22,22.5335,9.2,15,49.3106,39.17,16.850332,27.3413,...,NaN,36.854,51.5259,15.532262,25,17.6545,31.0393,2015,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2024-12-31 20:00:00,48.3,51.4,27.7,5.4,34.1,26.3,33.3,49.5,15.9,...,14.1,20.3,23.8,16.0,25.4,8.4,14.0,2024,NaN,NaN
35060,2024-12-31 21:00:00,67.0,65.5,19.9,7.7,34.6,27.5,34.0,48.7,16.1,...,12.6,20.1,28.4,19.3,25.0,12.8,12.7,2024,NaN,NaN
35061,2024-12-31 22:00:00,72.7,30.2,17.4,14.5,29.3,26.9,31.7,48.1,15.9,...,10.9,27.3,22.5,26.9,24.0,20.7,12.1,2024,NaN,NaN
35062,2024-12-31 23:00:00,79.9,36.0,23.6,NaN,16.0,23.5,26.8,38.4,12.5,...,10.2,26.4,25.3,23.0,23.6,13.4,12.7,2024,NaN,NaN


In [122]:
35064/4

8766.0

In [125]:
all_data

,Kod stacji,"('Jelenia Góra', 'DsJelGorOgin')","('Wrocław', 'DsWrocAlWisn')","('Wrocław', 'DsWrocWybCon')","('Bydgoszcz', 'KpBydPlPozna')","('Bydgoszcz', 'KpBydWarszaw')","('Lublin', 'LbLubObywate')","('Łódź', 'LdLodzCzerni')","('Zgierz', 'LdZgieMielcz')","('Zielona Góra', 'LuZielKrotka')",...,"('Przemyśl', 'PkPrzemGrunw')","('Gdańsk', 'PmGdaLeczkow')","('Kościerzyna', 'PmKosTargowa')","('Katowice', 'SlKatoKossut')","('Złoty Potok', 'SlZlotPotLes')","('Olsztyn', 'WmOlsPuszkin')","('Kalisz', 'WpKaliSawick')","('Szczecin', 'ZpSzczAndrze')","('Szczecin', 'ZpSzczPilsud')",Rok
0,2015-01-01 01:00:00.000,151.112,78,50,29.2,95.5,71.1012,59.73,58.690689,NaN,...,18.9027,NaN,NaN,51.389,49.9464,44.060833,25.5,NaN,NaN,2015
1,2015-01-01 02:00:00.000,262.566,42,33.8244,27.1,86.1,95.2582,185.92,44.501255,51.8628,...,15.9257,75.5278,NaN,37.709,45.6291,49.943329,32.5,81.8501,215.379,2015
2,2015-01-01 03:00:00.005,222.83,27,28.7215,7.9,23.8,49.5078,54.25,24.163937,15.866,...,16.1709,8.87389,NaN,41.984,49.2093,20.688448,24.3,25.7333,27.7458,2015
3,2015-01-01 04:00:00.010,210.767,24,20.6891,7.1,15.9,60.8468,50.15,19.516039,20.1187,...,16.0252,8.83194,22.7895,45.404,53.316,17.437717,24.4,18.624,26.5201,2015
4,2015-01-01 05:00:00.015,191.211,22,22.5335,9.2,15,49.3106,39.17,16.850332,27.3413,...,15.5498,6.5675,NaN,36.854,51.5259,15.532262,25,17.6545,31.0393,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2024-12-31 20:00:00,48.3,51.4,27.7,5.4,34.1,26.3,33.3,49.5,15.9,...,19.7,11.2,14.1,20.3,23.8,16.0,25.4,8.4,14.0,2024
35060,2024-12-31 21:00:00,67.0,65.5,19.9,7.7,34.6,27.5,34.0,48.7,16.1,...,19.3,14.0,12.6,20.1,28.4,19.3,25.0,12.8,12.7,2024
35061,2024-12-31 22:00:00,72.7,30.2,17.4,14.5,29.3,26.9,31.7,48.1,15.9,...,17.4,15.8,10.9,27.3,22.5,26.9,24.0,20.7,12.1,2024
35062,2024-12-31 23:00:00,79.9,36.0,23.6,NaN,16.0,23.5,26.8,38.4,12.5,...,15.1,16.1,10.2,26.4,25.3,23.0,23.6,13.4,12.7,2024


In [123]:
all_data[8800:]

,Kod stacji,"('Jelenia Góra', 'DsJelGorOgin')","('Wrocław', 'DsWrocAlWisn')","('Wrocław', 'DsWrocWybCon')","('Bydgoszcz', 'KpBydPlPozna')","('Bydgoszcz', 'KpBydWarszaw')","('Lublin', 'LbLubObywate')","('Łódź', 'LdLodzCzerni')","('Zgierz', 'LdZgieMielcz')","('Zielona Góra', 'LuZielKrotka')",...,"('Przemyśl', 'PkPrzemGrunw')","('Gdańsk', 'PmGdaLeczkow')","('Kościerzyna', 'PmKosTargowa')","('Katowice', 'SlKatoKossut')","('Złoty Potok', 'SlZlotPotLes')","('Olsztyn', 'WmOlsPuszkin')","('Kalisz', 'WpKaliSawick')","('Szczecin', 'ZpSzczAndrze')","('Szczecin', 'ZpSzczPilsud')",Rok
8800,2018-01-02 17:00:00.000,"8,60344","8,39689","8,67076","35,6325","57,2119","34,4","27,0","77,507","13,2562",...,"56,5114","41,1335","68,7199","33,6967","25,5095","45,6337",NaN,"15,2415","30,2299",2018
8801,2018-01-02 18:00:00.000,"5,58553","11,5106","10,2662","55,548","70,0692","43,1","16,0","78,932","10,9331",...,"47,3127","40,5368",NaN,"47,3631","27,7201","43,6108",NaN,"17,2885","25,166",2018
8802,2018-01-02 19:00:00.000,"7,8","7,5831","11,5","56,0308","54,7932","55,8","24,0","61,832","10,3739",...,"58,7425","47,2193","48,3378","24,0","33,9838","47,451",NaN,"21,3926","33,5007",2018
8803,2018-01-02 20:00:00.000,"9,91371","16,8657","12,7","44,2022",NaN,"54,6","31,0","47,582","9,03985",...,"60,107","38,8215",NaN,"14,0092","16,4202","49,396",NaN,"18,5841","27,0406",2018
8804,2018-01-02 21:00:00.000,"7,05287","10,6736","15,1122","39,1328",NaN,"62,0","14,0","43,307","6,32162",...,"47,8934","23,1279","80,4509","17,7075","13,6135","46,3593",NaN,"10,89","18,9513",2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2024-12-31 20:00:00,48.3,51.4,27.7,5.4,34.1,26.3,33.3,49.5,15.9,...,19.7,11.2,14.1,20.3,23.8,16.0,25.4,8.4,14.0,2024
35060,2024-12-31 21:00:00,67.0,65.5,19.9,7.7,34.6,27.5,34.0,48.7,16.1,...,19.3,14.0,12.6,20.1,28.4,19.3,25.0,12.8,12.7,2024
35061,2024-12-31 22:00:00,72.7,30.2,17.4,14.5,29.3,26.9,31.7,48.1,15.9,...,17.4,15.8,10.9,27.3,22.5,26.9,24.0,20.7,12.1,2024
35062,2024-12-31 23:00:00,79.9,36.0,23.6,NaN,16.0,23.5,26.8,38.4,12.5,...,15.1,16.1,10.2,26.4,25.3,23.0,23.6,13.4,12.7,2024


In [86]:
data = all_data.copy()
import pandas as pd
import numpy as np
# ensure strings
s = data["Kod stacji"].astype(str)

# first attempt: with milliseconds
dt = pd.to_datetime(s, format="%Y-%m-%d %H:%M:%S.%f", errors="coerce")

# second attempt: without milliseconds, only where first failed
mask = dt.isna()
dt[mask] = pd.to_datetime(s[mask], format="%Y-%m-%d %H:%M:%S", errors="coerce")

# assign back
data["Kod stacji"] = dt

# extract year and month
data["year"] = dt.dt.year
data["month"] = dt.dt.month

data.drop("Rok", axis=1, inplace=True)

data

,Kod stacji,"('Jelenia Góra', 'DsJelGorOgin')","('Wrocław', 'DsWrocAlWisn')","('Wrocław', 'DsWrocWybCon')","('Bydgoszcz', 'KpBydPlPozna')","('Bydgoszcz', 'KpBydWarszaw')","('Lublin', 'LbLubObywate')","('Łódź', 'LdLodzCzerni')","('Zgierz', 'LdZgieMielcz')","('Zielona Góra', 'LuZielKrotka')",...,"('Gdańsk', 'PmGdaLeczkow')","('Kościerzyna', 'PmKosTargowa')","('Katowice', 'SlKatoKossut')","('Złoty Potok', 'SlZlotPotLes')","('Olsztyn', 'WmOlsPuszkin')","('Kalisz', 'WpKaliSawick')","('Szczecin', 'ZpSzczAndrze')","('Szczecin', 'ZpSzczPilsud')",year,month
0,2015-01-01 01:00:00.000,151.112,78,50,29.2,95.5,71.1012,59.73,58.690689,NaN,...,NaN,NaN,51.389,49.9464,44.060833,25.5,NaN,NaN,2015,1
1,2015-01-01 02:00:00.000,262.566,42,33.8244,27.1,86.1,95.2582,185.92,44.501255,51.8628,...,75.5278,NaN,37.709,45.6291,49.943329,32.5,81.8501,215.379,2015,1
2,2015-01-01 03:00:00.005,222.83,27,28.7215,7.9,23.8,49.5078,54.25,24.163937,15.866,...,8.87389,NaN,41.984,49.2093,20.688448,24.3,25.7333,27.7458,2015,1
3,2015-01-01 04:00:00.010,210.767,24,20.6891,7.1,15.9,60.8468,50.15,19.516039,20.1187,...,8.83194,22.7895,45.404,53.316,17.437717,24.4,18.624,26.5201,2015,1
4,2015-01-01 05:00:00.015,191.211,22,22.5335,9.2,15,49.3106,39.17,16.850332,27.3413,...,6.5675,NaN,36.854,51.5259,15.532262,25,17.6545,31.0393,2015,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2024-12-31 20:00:00.000,48.3,51.4,27.7,5.4,34.1,26.3,33.3,49.5,15.9,...,11.2,14.1,20.3,23.8,16.0,25.4,8.4,14.0,2024,12
35060,2024-12-31 21:00:00.000,67.0,65.5,19.9,7.7,34.6,27.5,34.0,48.7,16.1,...,14.0,12.6,20.1,28.4,19.3,25.0,12.8,12.7,2024,12
35061,2024-12-31 22:00:00.000,72.7,30.2,17.4,14.5,29.3,26.9,31.7,48.1,15.9,...,15.8,10.9,27.3,22.5,26.9,24.0,20.7,12.1,2024,12
35062,2024-12-31 23:00:00.000,79.9,36.0,23.6,NaN,16.0,23.5,26.8,38.4,12.5,...,16.1,10.2,26.4,25.3,23.0,23.6,13.4,12.7,2024,12


In [120]:
meta_cols = {"Kod stacji", "year", "month"}
station_cols = [c for c in data.columns if c not in meta_cols]

no_metadata_df = data.drop("Kod stacji", axis=1)

long = no_metadata_df.melt(id_vars=["year", "month"], value_vars=station_cols, var_name="station", value_name="pm2.5")
#long["station"].iloc[3]

long["city"] = long["station"].str.extract(r"'([^']+)'")
long.drop("station", axis=1, inplace=True)


## perform the actual aggregation
long["pm2.5"] = pd.to_numeric(long["pm2.5"], errors="coerce")
monthly_avg = long.groupby(["year", "month", "city"], as_index=False).mean(numeric_only=True)
(monthly_avg
.pivot(
    index=["year", "month"],
    columns="city",
    values="pm2.5"
)
.sort_index())

city        Białystok  Bydgoszcz     Gdańsk  Jelenia Góra     Kalisz  \
year month                                                             
2015 1      22.464238  23.289532  15.251184     24.504821  32.039886   
     2      41.037325  47.323201  21.435362     41.859821  48.900846   
     3      30.945504  30.459710  17.027339     28.603600  41.674071   
     4      13.426136  16.357370  10.397704     12.323373  24.054884   
     5      10.741935  12.749093   8.397277      8.716385  19.572950   
     6       9.250729   9.882780   7.627350      7.791419  15.819560   
     7       8.619519   9.943245   7.988024      6.879472  15.889289   
     8      13.242637  14.249278  10.970232     11.131890  22.185033   
     9      14.042398  14.226358   9.395071      7.141748  17.613177   
     10     26.063380  35.336876  19.428102     23.644826  42.418469   
     11     22.105183  33.002864  13.127917     25.710537  41.132756   
     12     25.110811  25.618770  14.048379     22.893786  26.462317   
2018 1            NaN        NaN        NaN           NaN        NaN   
     2            NaN        NaN        NaN           NaN        NaN   
     3            NaN        NaN        NaN           NaN        NaN   
     4            NaN        NaN        NaN           NaN        NaN   
     5            NaN        NaN        NaN           NaN        NaN   
     6            NaN        NaN        NaN           NaN        NaN   
     7            NaN        NaN        NaN           NaN        NaN   
     8            NaN        NaN        NaN           NaN        NaN   
     9            NaN        NaN        NaN           NaN        NaN   
     10           NaN        NaN        NaN           NaN        NaN   
     11           NaN        NaN        NaN           NaN        NaN   
     12           NaN        NaN        NaN           NaN        NaN   
2021 1      27.469003  24.913865  26.423242     29.901718  30.169934   
     2      31.066265  38.562274  34.452908     34.237084  40.491779   
     3      17.737550  19.692003  20.398686     26.880032  29.237250   
     4      13.555866  12.289326  14.588983     16.038263  19.031608   
     5       9.172043   6.832432  10.837720      8.564051   9.912900   
     6      10.835926   8.183610  12.157522     12.918083  13.436213   
     7      11.773700  10.635105  12.578711     10.734240  14.387558   
     8       8.046259   6.835615   8.882545      8.095207  11.490622   
     9       9.687065   9.411564  11.963975     12.849123  14.973283   
     10     16.687237  15.231369  17.760616     19.165852  20.810373   
     11     18.781944  16.832301  17.257728     26.115729  28.850152   
     12     17.814614  23.286439  20.018332     35.768634  39.478690   
2024 1      15.414013  18.802209  16.550539     23.274866  24.507392   
     2      11.007310  15.483306  15.185036     13.506178  18.619253   
     3      17.306864  23.233382  22.579217     19.934320  28.021131   
     4      10.525000  10.856421  15.131198      9.886528  13.471250   
     5      10.610215  10.213454  14.450538      8.234677  12.682661   
     6      10.507463   9.019757  12.335188      9.513333  12.278333   
     7       8.052414   7.740427   9.763795      7.453763  10.261725   
     8      10.822191   9.195042  11.628857      9.372581  12.873387   
     9      13.641168  13.426847  16.022270     11.491155  17.357799   
     10     12.582111  15.362927  14.074832     13.677898  19.584005   
     11     14.239888  17.426900  13.743611     20.962500  21.465132   
     12     13.630972  16.538968  13.868102     24.602957  27.353763   

city         Katowice  Kościerzyna     Kraków  Kędzierzyn-Koźle  Legionowo  \
year month                                                                   
2015 1      31.247839    36.803262  41.882353         31.394930  34.321645   
     2      56.355432    51.354948  66.885357         57.073201  62.339970   
     3      39.389542    33.843097  50.628948         38.120842  56.189266   
     4   

In [ ]:
def monthly_average(data, metadata_idx=3):
    """(Zad2)
    Function used to compute monthly averages of PM2.5 concentration in Zad2.
    Averages over measurements in all stations for a given city in a given month (in a given year)
    Args:
        data (pandas.DataFrame): a dataframe of PM2.5 levels
        metadata_idx (int): index of the first non-metadata row in the `data` DataFrame

    Returns:
        result (pandas.DataFrame): a dataframe of average monthly PM2.5 in each city with MultiIndex (year, month) and cities as columns.
    """
    # add "year" an "month" columns for downstream indexing
    dt = pd.to_datetime(
        data["Miejscowość"],
        format="%Y-%m-%d %H:%M:%S",
        errors="coerce"
    )

    data["year"] = dt.dt.year
    data["month"] = dt.dt.month

    ## prepare the new "city" column (used for aggregation)
    meta_cols = {"Miejscowość", "year", "month"}
    station_cols = [c for c in data.columns if c not in meta_cols]

    # construct a dictionary for mapping stations to cities
    city_names = (
        pd.Series(station_cols)
        .str.replace(r"\.\d+$", "", regex=True) # this converts eg. "Kraków.1" to "Kraków"
    )

    station_to_city = dict(zip(station_cols, city_names))

    # drop metadata, then melt
    no_metadata_df = data[metadata_idx:].drop("Miejscowość", axis=1)

    long = no_metadata_df.melt(id_vars=["year", "month"], value_vars=station_cols, var_name="station", value_name="pm2.5")

    # ensure all stations are assigned to a city
    long["city"] = long["station"].map(station_to_city)
    long.drop("station", axis=1, inplace=True)

    ## perform the actual aggregation
    long["pm2.5"] = pd.to_numeric(long["pm2.5"], errors="coerce")
    monthly_avg = long.groupby(["year", "month", "city"], as_index=False).mean(numeric_only=True)

    # pivot back to a readable format
    result = (
        monthly_avg
        .pivot(
            index=["year", "month"],
            columns="city",
            values="pm2.5"
        )
        .sort_index()
    )

    # convert Multindex from float to int
    idx = result.index

    idx = idx.set_levels(
        idx.levels[idx.names.index("year")].astype(int),
        level="year"
    )

    idx = idx.set_levels(
        idx.levels[idx.names.index("month")].astype(int),
        level="month"
    )

    result.index = idx

    return result

mean_df = monthly_average(data)
mean_df

### Zadanie 2

Poniższy kod wykonuje następujące zadania:
 - Obliczanie średniego miesięcznego stężenia PM2.5 dla każdej stacjii i roku.
 - Rysowanie wykresu liniowego miesięcznych wartości stężenia PM2.5 w **2015** i 2024 roku dla Warszawy i Katowic.

In [ ]:
data = pd.read_csv("all_data.csv")
mean_df = monthly_average(data)

# FIXME: ZMIENIĆ NA 2015!
plot = plot_city_trends(mean_df, years=[2014, 2024])
plt.show(plot)

ValueError: None of the requested cities ('Warszawa', 'Katowice') found in columns. Available cities: [('Białystok', 'PdBialUpalna'), ('Bydgoszcz', 'KpBydPlPozna'), ('Bydgoszcz', 'KpBydWarszaw'), ('Gdańsk', 'PmGdaLeczkow'), ('Jelenia Góra', 'DsJelGorOgin'), ('Kalisz', 'WpKaliSawick'), ('Katowice', 'SlKatoKossut'), ('Kościerzyna', 'PmKosTargowa'), ('Kraków', 'MpKrakAlKras'), ('Kraków', 'MpKrakBulwar'), ('Kędzierzyn-Koźle', 'OpKKozBSmial'), ('Legionowo', 'MzLegZegrzyn'), ('Lublin', 'LbLubObywate'), ('Olsztyn', 'WmOlsPuszkin'), ('Piastów', 'MzPiasPulask'), ('Przemyśl', 'PkPrzemGrunw'), ('Płock', 'MzPlocMiReja'), ('Radom', 'MzRadTochter'), ('Siedlce', 'MzSiedKonars'), ('Szczecin', 'ZpSzczAndrze'), ('Szczecin', 'ZpSzczPilsud'), ('Warszawa', 'MzWarAlNiepo'), ('Warszawa', 'MzWarWokalna'), ('Wrocław', 'DsWrocAlWisn'), ('Wrocław', 'DsWrocWybCon'), ('Zgierz', 'LdZgieMielcz'), ('Zielona Góra', 'LuZielKrotka'), ('Złoty Potok', 'SlZlotPotLes'), ('Łódź', 'LdLodzCzerni'), ('Żyrardów', 'MzZyraRoosev')

#### Interpretacja wyników **do poprawy (bo 2015 ma teraz być)**

Zarówno w Warszawie jak i w Katowicach średnie stężnie PM2.5 spadło między rokiem 2014 a 2024. W obydwu miastach poziom drobnych zanieczyszczeń jest większy w miesiącach zimowych niż w letnich. Podczas trwania sezonu grzewczego, poziom PM2.5 w danym miesiącu jest wyższy w Katowicach niż w Warszawie; poza sezonem grzewczym, zależność ta nie zachodzi.

W roku 2024, w obydwu miastach, różnice w stężeniu drobnego pyłu pomiedzy miesiącami letnimi i zimowymi są wyraźnie mniejsze niż w 2014. Być może wynika to z usprawnień w sposobie ogrzewania i termoizolacji budynków, które zostały wprowadzone w życie w ciągu tamtej dekady. 

In [ ]:
# **FIXME:** Napisać nowe wnioski dla danych z innych lat!!!
# Tutaj muszę poczekać, na poprawione zadanie 1

### Zadanie 3

In [ ]:
df = pd.read_csv("all_data.csv", header=None)
df.columns = df.iloc[0].astype(str).str.strip()
df = df[1:]

In [ ]:
#obliczanie średniej miesięcznej PM2.5 dla każdej miejscowości oraz filtrowanie miast, w których brakuje średniej miesięcznej
def monthly_avg(df):
    df = df.copy()
    df = df.drop(columns=['year'], errors='ignore')

    locations = df.columns[1:]

    def unique_loc(names):
        how_many = {}
        locs = []
        for n in names:
            if n not in how_many:
                how_many[n] = 1
                locs.append(n)
            else:
                how_many[n] += 1
                locs.append(f"{n}{how_many[n]}")
        return locs
    
    unique_cols = unique_loc(locations)
    df.columns = [df.columns[0]] + list(unique_cols)

    cols_locations = dict(zip(unique_cols, locations))

    df[df.columns[0]] = pd.to_datetime(df[df.columns[0]], format="%Y-%m-%d %H:%M:%S", errors='coerce')
    df = df.dropna(subset=[df.columns[0]]).reset_index(drop=True)

    for col in unique_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    long_format = df.melt(id_vars=df.columns[0], value_vars=unique_cols, var_name='station', value_name='pm25')
    long_format['location'] = long_format['station'].map(cols_locations)
    long_format['year'] = long_format[df.columns[0]].dt.year
    long_format['month'] = long_format[df.columns[0]].dt.month

    monthly = long_format.groupby(['location', 'year', 'month'], as_index=False)['pm25'].mean()

    all_locations = []
    for loc in monthly['location'].unique():
        df_loc = monthly[monthly['location'] == loc]
        if not df_loc['pm25'].isna().any():
            all_locations.append(loc)

    monthly_filtered = monthly[monthly['location'].isin(all_locations)].copy()

    return monthly_filtered

In [ ]:
monthly = monthly_avg(df)
locations = monthly['location'].unique()
zmin = monthly['pm25'].min()
zmax = monthly['pm25'].max()
years = [2014, 2019, 2024]

n = len(locations)
rows = int(np.ceil(n / 2))
cols = 2

fig = make_subplots(rows=rows, cols=cols, subplot_titles=[f"{loc}" for loc in locations])
colorscale = "Viridis"

for i, loc in enumerate(locations):
    row = i // 2 + 1
    col = i % 2 + 1

    dfloc = monthly[(monthly['location'] == loc) & (monthly['year'].astype(int).isin(years))]

    heatmap_data = dfloc.pivot(index='year', columns='month', values='pm25')
    y = heatmap_data.index.astype(str)[::-1]

    showscale = True if i == 0 else False

    hm = go.Heatmap(z=heatmap_data.values[::-1, :], x=heatmap_data.columns, y=y,
                    colorscale=colorscale,
                    zmin=zmin, zmax=zmax,
                    colorbar=dict(title=dict(text="PM2.5 µg/m³"),
                                tickmode="array",
                                tickvals=np.linspace(zmin, zmax, 5),
                                ticktext=[f"{v:.0f}" for v in np.linspace(zmin, zmax, 5)],
                                len=0.2,
                                y=0.7,
                                x=1.05),
                    hovertemplate="Rok: %{y}<br>Miesiąc: %{x}<br>PM2.5: %{z} µg/m³",
                    showscale=showscale)

    fig.add_trace(hm, row=row, col=col)

fig.update_xaxes(tickmode="array", tickvals=list(range(1, 13)), ticktext=list(range(1, 13)))

for i in range(1, rows*cols + 1):
    fig.update_yaxes(categoryorder='array',
                    categoryarray=[str(y) for y in years],
                    autorange="reversed",
                    row=(i-1) // 2 + 1,
                    col=(i-1) % 2 + 1)

fig.update_layout(height=350 * rows, width=1000, title=dict(text='Średnie PM2.5 w latach 2014, 2019 i 2024', x=0.5, y=0.99), font=dict(size=12))

fig.show()

#### Interpretacja powyższych obserwowanych wyników
W każdym roku można zaobserwować wyższe stężenia PM2.5 w okresie zimowym oraz niższe latem. Jak można byłoby się spodziewać, w większych miastach wartości PM2.5 są wyższe niezależnie od pory roku. Jednocześnie widoczna jest wyraźna tendencja spadkowa między analizowanymi latami, co wskazuje na poprawę jakości powietrza w danych miejscowościach.

### Brudnopis

### Zadanie 4

Poniższy kod wykonuje następujące zadania:
 - Zliczanie dni, w których została przekroczona norma PM2.5 w danej stacji pomiarowej, w danym roku.

 - Identyfikacja trzech stacji z największą oraz trzech stacji z najmniejszą liczbą takich dni w 2024 roku.

 - Rysowanie wykresu typu *grouped barplot* poziomów stężenia PM2.5 dla 6. zidentyfikowanych stacji w latach 2015, 2018, 2021 i 2024.

In [ ]:
who_threshold = 15
metadata_idx = 3
top_n = 3


all_data = pd.read_csv("./all_data.csv", header=0)

exceedance_counts = count_days_over_treshold(data=all_data,treshold=who_threshold)
barplot = plot_pm25_exceedance_bars(exceedance_counts, top_n=top_n, base_year=2024, threshold=who_threshold) 

plt.show(barplot)

In [ ]:
# **FIXME:** Napisać nowe wnioski dla danych z innych lat!!!
# 2015, 2018, 2021 i 2024
# Tutaj muszę poczekać, na poprawione zadanie 1

#### Interpretacja wyników **do poprawy (ma być 2015, 2018, 2021 i 2024)**

Powyższy wykres pokazuje liczbę dni, w których średnie dobowe stężenie PM2.5 przekroczyło wartość 15 µg/m³ dla sześciu wybranych stacji - trzech z najmniejszą (wykresy 1-3) i trzech z największą (wykresy 4-6) liczbą przekroczeń w 2024 r. Stacje z największą liczbą przekroczeń wykazują wyraźnie gorszą jakość powietrza w niemal wszystkich porównywanych latach, co sugeruje trwałe źródła emisji lub niekorzystne warunki meteorologiczne.

Zgodnie z tym co zauwożono w poprzednich zadaniach, stężenie PM2.5 zmierzone przez daną stację ma trend spadkowy w czasie.

### Ogólna interpretacja obserwowanych wyników **do poprawy (ma być 2015, 2018, 2021 i 2024)**

Średnie miesięczne wartości PM2.5 w latach 2014, 2019 i 2024 wskazuje na tendencję spadkową zanieczyszczenia powietrza w badanym okresie. Najwyższe wartości można zaobserwować w większych miastach, co może wiązać się z większym natężeniem ruchu drogowego i działalności przemysłowej. Dodatkowo dane wskazują na zależność stężenia PM2.5 od pory roku - wartości są najniższe w  miesiącach letnich, a wzrastają w sezonie zimowym, co prawdopodobnie wynika z ogrzewania budynków i wcześniej wspomnianym większym natężeniem ruchu drogowego. Średnie miesięczne wartości PM2.5 są zatem ściśle związane z działalnością człowieka i porą roku.